In [1]:
import pandas as pd

# Cargar el dataset de películas
movie_dataset_cleaned = pd.read_csv('D:/SOYHENRY_PI_1/fastapi_project/movie_dataset_cleaned.csv')

# Cargar el dataset con información del reparto y equipo de producción
df_unido = pd.read_csv('D:/SOYHENRY_PI_1/fastapi_project/df_unido.csv')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11080\3151750402.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_dataset_cleaned = pd.read_csv('D:/SOYHENRY_PI_1/fastapi_project/movie_dataset_cleaned.csv')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11080\3151750402.py:7: DtypeWarning: Columns (9,10,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unido = pd.read_csv('D:/SOYHENRY_PI_1/fastapi_project/df_unido.csv')


In [2]:
# Echa un vistazo a las primeras filas de ambos DataFrames
print(movie_dataset_cleaned.head())

       budget                                             genres     id  \
0  30000000.0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862   
1  65000000.0  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844   
2         0.0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602   
3  16000000.0  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357   
4         0.0                     [{'id': 35, 'name': 'Comedy'}]  11862   

  original_language                                           overview  \
0                en  Led by Woody, Andy's toys live happily in his ...   
1                en  When siblings Judy and Peter discover an encha...   
2                en  A family wedding reignites the ancient feud be...   
3                en  Cheated on, mistreated and stepped on, the wom...   
4                en  Just when George Banks has recovered from his ...   

  popularity                               production_companies  \
0  21.946943     [{'name': 'Pixar Ani

In [3]:
# Echa un vistazo a las primeras filas de ambos DataFrames
print(df_unido.head())


        id  cast_id                character                 credit_id  \
0    862.0     14.0            Woody (voice)  52fe4284c3a36847f8024f95   
1   8844.0     15.0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99   
2  15602.0     16.0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d   
3  31357.0     17.0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1   
4  11862.0     18.0              Rex (voice)  52fe4284c3a36847f8024fa5   

   gender     id.1           name  order                      profile_path  \
0     2.0     31.0      Tom Hanks    0.0  /pQFoyx7rp09CJTAb932F2g8Nlho.jpg   
1     2.0  12898.0      Tim Allen    1.0  /uX2xVf6pMmPepxnvFWyBtjexzgY.jpg   
2     2.0   7167.0    Don Rickles    2.0  /h5BcaDMPRVLHLDzbQavec4xfSdt.jpg   
3     2.0  12899.0     Jim Varney    3.0  /eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg   
4     2.0  12900.0  Wallace Shawn    4.0  /oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg   

                credit_id.1 department  gender.1     id.2         job  \
0  52fe4284c3